In [1]:
{-# LANGUAGE OverloadedStrings #-}

In [2]:
import Network.HTTP.Req
import qualified Data.ByteString as B
import Data.Default.Class
import Control.Monad.IO.Class
import Data.Aeson
import Data.Monoid ((<>))
import qualified Data.Vector as V
import qualified Data.HashMap.Strict as H
import qualified Data.Text as T

email <- B.init <$> B.readFile "./email"
key   <- B.init <$> B.readFile "./api-key"

emailT <- T.pack . init <$> readFile "./email"

narrow :: Value
narrow = Array $ V.fromList
    [ Object $ H.fromList [("operator", "sender"), ("operand", String emailT)]
    , Object $ H.fromList [("operator", "stream"), ("operand", "music")]
    ]

params
    =  "anchor"     =: (0  :: Int)
    <> "num_before" =: (0  :: Int)
    <> "num_after"  =: (1  :: Int)
--    <> "narrow"     =: (narrow :: Value)

param = "stream" =: "music"


r = req GET (https "recurse.zulipchat.com" /: "api" /: "v1" /: "messages") NoReqBody jsonResponse ((basicAuth email key) <> params)

r' = req GET
    (https "recurse.zulipchat.com" /: "api" /: "v1" /: "get_stream_id")
    NoReqBody jsonResponse
    ((basicAuth email key) <> param)

-- res <- responseBody <$> runReq def r :: IO Value
-- res